# Bash environment variables 

In [ ]:
%%bash
cat <<EOF > env.sh
# =============================
# BIDS environment setup
# =============================

export PROJECT_ROOT="/mnt/c/Users/hapla/Documents/GitHub/albert-plante_project"
export BIDS_ROOT="\${PROJECT_ROOT}/data"
export SUBJECT="0204"

# =============================
# Derived and temp directories
# =============================

export WORKDIR="\${BIDS_ROOT}/derivatives/tmp_\${SUBJECT}"
export RESULTSDIR="\${PROJECT_ROOT}/results/results_\${SUBJECT}"
export TEMPLATE_DIR="\${BIDS_ROOT}/derivatives/templates"


# =============================
# Input images
# =============================

export MP2RAGE="\${BIDS_ROOT}/sub-\${SUBJECT}/anat/sub-\${SUBJECT}_T1w.nii.gz"
export DWI="\${BIDS_ROOT}/sub-\${SUBJECT}/dwi/sub-\${SUBJECT}_dwi.nii.gz"
export rDWI="\${BIDS_ROOT}/derivatives/registered_dwi/sub-\${SUBJECT}/dwi/sub-\${SUBJECT}_dwi.nii.gz"
export T1_MAP="\${BIDS_ROOT}/derivatives/t1_map/sub-\${SUBJECT}/sub-\${SUBJECT}_t1_map.nii.gz"
export SEG_DWI="\${BIDS_ROOT}/derivatives/optic_nerve_segmentations/sub-\${SUBJECT}/dwi/sub-\${SUBJECT}_label-ON_seg.nii.gz"
export SEG_MP2RAGE="\${BIDS_ROOT}/derivatives/optic_nerve_segmentations/sub-\${SUBJECT}/anat/sub-\${SUBJECT}_label-ON_seg.nii.gz"
export FA_MAP="\${BIDS_ROOT}/derivatives/fa_map/sub-\${SUBJECT}/sub-\${SUBJECT}_scanner_FA.nii.gz"


# =============================
# Templates / Atlases
# =============================

export MNI="\${TEMPLATE_DIR}/MNI152_T1_1mm.nii.gz"
export ATLAS="\${TEMPLATE_DIR}/Atlas_MNI.nii.gz"
export MNI_LABEL="\${TEMPLATE_DIR}/Optic_Nerve_Label_MNI.gz"

# =============================
# Registered outputs (generated)
# =============================

export DWI_TO_MP2RAGE="\${WORKDIR}/dwi2mp2rage"
export MP2RAGE_TO_MNI="\${WORKDIR}/mp2rage2mni"
export DWI_MNI="\${WORKDIR}/dwi_mni.nii.gz"
export SEG_DWI_TO_MP2RAGE="\${WORKDIR}/seg_dwi_to_mp2rage.nii.gz"
export SEG_DWI_TO_MNI="\${WORKDIR}/seg_dwi_to_mni.nii.gz"
export SEG_MP2RAGE_TO_MNI="\${WORKDIR}/seg_mp2rage_to_mni.nii.gz"
export T1_MAP_TO_MNI="\${WORKDIR}/t1_map_to_mni.nii.gz"
export MP2RAGE_MNI_ROI="\${WORKDIR}/mp2rage_mni_roi.nii.gz"
export DWI_MNI_ROI="\${WORKDIR}/dwi_mni_roi.nii.gz"
export CORRECTED_DWI="\${WORKDIR}/dwi_mni_corrected.nii.gz"
export FINAL_SEG_DWI="\${WORKDIR}/seg_dwi_mni_corrected.nii.gz"
export FA_TO_MP2RAGE="\${WORKDIR}/fa2mp2rage.nii.gz"
export FA_TO_MNI="\${WORKDIR}/fa2mni.nii.gz"
export CORRECTED_FA="\${WORKDIR}/fa2mni_corrected.nii.gz"

# =============================
# Results outputs (generated)
# =============================

export MP2RAGE_MNI="\${RESULTSDIR}/mp2rage_mni.nii.gz"
export DWI_MNI="\${RESULTSDIR}/dwi_mni.nii.gz"
export SEG_MP2RAGE_MNI="\${RESULTSDIR}/seg_mp2rage_mni.nii.gz"
export SEG_DWI_MNI="\${RESULTSDIR}/seg_dwi_mni.nii.gz"
export T1_MAP_MNI="\${RESULTSDIR}/t1_map_mni.nii.gz"
export FA_MAP_MNI="\${RESULTSDIR}/fa_map_mni.nii.gz"
EOF

source ./env.sh
mkdir -p "$WORKDIR"
mkdir -p "$RESULTSDIR"

# Image Registration

**Diffusion b0 image to MP2RAGE (Rigid Registration)**

Because DWI are normally slabs rather than full brain image, the next registration may not always work reliably. Visually check the output, and if it's offset, rerun the registration.

In [ ]:
%%bash
source env.sh
export PATH=$HOME/ants/bin:$PATH

# DWI to MP2RAGE
antsRegistrationSyN.sh \
  -d 3 -t r \
  -f "$MP2RAGE" \
  -m "$rDWI" \
  -o "$DWI_TO_MP2RAGE"

**MP2RAGE to MNI 152 (Affine Registration)**

In [ ]:
%%bash
source env.sh
export PATH=$HOME/ants/bin:$PATH

# MP2RAGE to MNI 
antsRegistrationSyN.sh \
  -d 3 -t a \
  -f "$MNI" \
  -m "$MP2RAGE" \
  -o "$MP2RAGE_TO_MNI"

**Diffusion and MP2RAGE derivatives to MNI 152**

In [ ]:
%%bash
source env.sh
export PATH=$HOME/ants/bin:$PATH

# DWI to MNI (via MP2RAGE to MNI Matrice)
antsApplyTransforms \
  -d 3 \
  -i "${DWI_TO_MP2RAGE}Warped.nii.gz" \
  -r "$MNI" \
  -o "$DWI_MNI" \
  -n NearestNeighbor \
  -t "${MP2RAGE_TO_MNI}0GenericAffine.mat"

# DWI Segmentation to MP2RAGE (via DWI to MP2RAGE)
antsApplyTransforms \
  -d 3 \
  -i "$SEG_DWI" \
  -r "$MP2RAGE" \
  -o "$SEG_DWI_TO_MP2RAGE" \
  -n NearestNeighbor \
  -t "${DWI_TO_MP2RAGE}0GenericAffine.mat"

# DWI Segmentation to MNI (via MP2RAGE to MNI)
antsApplyTransforms \
  -d 3 \
  -i "$SEG_DWI_TO_MP2RAGE" \
  -r "$MNI" \
  -o "$SEG_DWI_TO_MNI" \
  -n NearestNeighbor \
  -t "${MP2RAGE_TO_MNI}0GenericAffine.mat" 


# MP2RAGE Segmentation to MNI (via MP2RAGE to MNI)
antsApplyTransforms \
  -d 3 \
  -i "$SEG_MP2RAGE" \
  -r "$MNI" \
  -o "$SEG_MP2RAGE_TO_MNI" \
  -n NearestNeighbor \
  -t "${MP2RAGE_TO_MNI}0GenericAffine.mat"

# T1 MAP to MNI (via MP2RAGE to MNI)
antsApplyTransforms \
  -d 3 \
  -i "$T1_MAP" \
  -r "$MNI" \
  -o "$T1_MAP_TO_MNI" \
  -n NearestNeighbor \
  -t "${MP2RAGE_TO_MNI}0GenericAffine.mat"

# DWI FA to MP2RAGE (via DWI to MP2RAGE)
antsApplyTransforms \
  -d 3 \
  -i "$FA_MAP" \
  -r "$MP2RAGE" \
  -o "$FA_TO_MP2RAGE" \
  -n NearestNeighbor \
  -t "${DWI_TO_MP2RAGE}0GenericAffine.mat"

# DWI FA to MNI (via MP2RAGE to MNI)
antsApplyTransforms \
  -d 3 \
  -i "$FA_TO_MP2RAGE" \
  -r "$MNI" \
  -o "$FA_TO_MNI" \
  -n NearestNeighbor \
  -t "${MP2RAGE_TO_MNI}0GenericAffine.mat" 

**ROI creation using MNI 152 standardized coordinates**

In [ ]:
%%bash
source env.sh
fslroi "${MP2RAGE_TO_MNI}Warped.nii.gz" "$MP2RAGE_MNI_ROI" 40 101 140 60 22 25
fslroi "$DWI_MNI" "$DWI_MNI_ROI" 40 101 140 60 22 25

**Diffusion to MP2RAGE (ROI Rigid Registration)**

In [ ]:
%%bash
source env.sh
export PATH=$HOME/ants/bin:$PATH

# DWI to MP2RAGE (Rigid Registration)
antsRegistrationSyN.sh \
  -d 3 -t r \
  -f "$MP2RAGE_MNI_ROI" \
  -m "$DWI_MNI_ROI" \
  -o "${WORKDIR}/roi2roi_"

**Diffusion derivatives to MP2RAGE (via ROI)**

In [ ]:
%%bash
source env.sh
export PATH=$HOME/ants/bin:$PATH

# Corrected DWI to MNI (via ROI DWI to ROI MP2RAGE Matrice)
antsApplyTransforms \
  -d 3 \
  -i "$DWI_MNI" \
  -r "${MP2RAGE_TO_MNI}Warped.nii.gz" \
  -o "$CORRECTED_DWI" \
  -n NearestNeighbor \
  -t "${WORKDIR}/roi2roi_0GenericAffine.mat"

# DWI Segmentation to MP2RAGE (via ROI DWI to ROI MP2RAGE Matrice)
antsApplyTransforms \
  -d 3 \
  -i "$SEG_DWI_TO_MNI" \
  -r "${MP2RAGE_TO_MNI}Warped.nii.gz" \
  -o "$FINAL_SEG_DWI" \
  -n NearestNeighbor \
  -t "${WORKDIR}/roi2roi_0GenericAffine.mat"

# Corrected FA Map to MNI (via ROI DWI to ROI MP2RAGE Matrice)
antsApplyTransforms \
  -d 3 \
  -i "$FA_TO_MNI" \
  -r "${MP2RAGE_TO_MNI}Warped.nii.gz" \
  -o "$CORRECTED_FA" \
  -n NearestNeighbor \
  -t "${WORKDIR}/roi2roi_0GenericAffine.mat"